In [ ]:
!pip install pydicom==2.0.0
!pip install pymedphys==0.34.0dev0

In [ ]:
from urllib import request

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom
import pymedphys

# Private imports, not part of the stable API
from pymedphys._dicom.dose import get_dose_grid_structure_mask
from pymedphys._dicom.structure import pull_structure

In [ ]:
dose_url = 'https://zenodo.org/record/4042842/files/RD.2.16.840.1.114337.1.1.1600065398.2_Anonymised.dcm?download=1'
structure_url = 'https://zenodo.org/record/4042842/files/RS.1.2.840.10008.5.1.4.1.1.481.3.1600145017_Anonymised.dcm?download=1'

dose_filepath = 'dose.dcm'
structure_filepath = 'structure.dcm'

In [ ]:
request.urlretrieve(dose_url, dose_filepath)
request.urlretrieve(structure_url, structure_filepath)

In [ ]:
dose_dataset = pydicom.read_file(dose_filepath, force=True)
structure_dataset = pydicom.read_file(structure_filepath, force=True)

In [ ]:
structure_names_to_pull = ['11', '12', '13', '14', '15', '16', '17', '18']

In [ ]:
(dose_z, dose_y, dose_x), dose = pymedphys.dicom.zyx_and_dose_from_dataset(dose_dataset)

xx, yy = np.meshgrid(dose_x, dose_y)

In [ ]:
mask = get_dose_grid_structure_mask('11', structure_dataset, dose_dataset)

In [ ]:
structure_x, structure_y, structure_z = pull_structure('11', structure_dataset)

In [ ]:
structure_index = 0

z_slice = structure_z[structure_index][0]
z_slice

In [ ]:
dose_z_index = np.where(dose_z==z_slice)[0][0]
dose_z_index

In [ ]:
plt.scatter(xx, yy, c=mask[dose_z_index, :, :])
plt.plot(structure_x[structure_index], structure_y[structure_index])

plt.axis('equal')
plt.ylim([-240, -220])
plt.xlim([-20, 0])

In [ ]:
dose[mask]

In [ ]:
np.mean(dose[mask])

In [ ]:
np.min(dose[mask])

In [ ]:
np.max(dose[mask])

In [ ]:
np.std(dose[mask])

In [ ]:
table_data = {}

for structure_name in structure_names_to_pull:

    mask = get_dose_grid_structure_mask(structure_name, structure_dataset, dose_dataset)
    _, dose = pymedphys.dicom.zyx_and_dose_from_dataset(dose_dataset)
    
    masked_dose = dose[mask]
    
    table_data[structure_name] = {}
    
    table_data[structure_name]['mean'] = np.mean(masked_dose)
    table_data[structure_name]['min'] = np.min(masked_dose)
    table_data[structure_name]['max'] = np.max(masked_dose)
    table_data[structure_name]['std'] = np.std(masked_dose)

In [ ]:
table_data

In [ ]:
df = pd.DataFrame.from_dict(table_data)
df = df.round(2)
df

In [ ]:
df.to_csv('structure_stats.csv')